In [1]:
from dotenv import load_dotenv
import os, requests, sys, json
from IPython.display import JSON
from pathlib import Path
import logging.handlers
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.signal import convolve2d
import warnings
import seaborn as sns

from os import listdir
from os.path import isfile, join
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Helpers.Exception_Handling import Exception_Handling as eh
from Helpers.Mongo_Interface import Mongo_Interface as mi

env_path = Path(fr"{os.environ['USERPROFILE']}\.env") 
load_dotenv(dotenv_path=env_path)

True

In [2]:
path = r'N:\connectxdocs'
onlyfiles = os.listdir(path)

In [3]:
print(len(onlyfiles))
onlyfiles[0:5]

223887


['45907088.json',
 '45907087.json',
 '45907086.json',
 '45907085.json',
 '45907084.json']

In [4]:
import threading

In [28]:
thread_manager = {}
thread_limit = 100
results = []

import time
def second_thread(file_name):
    global results
    document = json.load(open(f"{path}\\{file_name}"))
    results.append(document)

for i in range(thread_limit):
    thread_manager[i] = None

for file_name in tqdm(onlyfiles):
    while True:
        thread_started = False
        for i in range(thread_limit):
            if thread_manager[i] == None or not thread_manager[i].is_alive():
                thread_manager[i] = threading.Thread(target=second_thread, args=(file_name,))
                thread_manager[i].start()
                thread_started = True
                break
        if thread_started:
            break
        
len(results)

 43%|███████████████████████████████▉                                          | 96794/223887 [02:40<03:31, 602.05it/s]


KeyboardInterrupt: 

In [29]:


with open(r"N:\documents.json", "w") as outfile:
    outfile.write(json.dumps({'documents':results}))

KeyboardInterrupt: 

In [ ]:
doc = {}
with open(r"N:\documents.json", "r") as outfile:
    doc = json.load(outfile)
doc['documents'][0]

In [30]:
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

In [32]:
columns = [item for item in range(1, 43)]
columns.append('board_series')
columns.append('winner')
columns.append('looser')
columns.append('is_stale')


df = pd.DataFrame(columns=columns)
obj =''
rows = []

for i in tqdm(range(len(results))):
    flat_board = results[i]['steps'][-1:][0][0]['observation']['board'].copy()
    if len(results[i]['steps']) < 3:
           continue
    board = np.resize(flat_board, (6,7))
    flat_board.append(results[i]['steps'])
    if winning_move(board, 1):
        flat_board.append(1)
        flat_board.append(2)
        flat_board.append(False)
    elif winning_move(board, 2):
        flat_board.append(2)
        flat_board.append(1)
        flat_board.append(False)
    else:
        flat_board.append(None)
        flat_board.append(None)
        flat_board.append(True)
    rows.append(flat_board)
    
df2 = pd.DataFrame(data=rows, columns=columns)
df = pd.concat([df, df2], ignore_index=True)
df

100%|█████████████████████████████████████████████████████████████████████████| 96794/96794 [00:03<00:00, 25840.50it/s]


,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
0,0,0,1,1,2,0,1,0,0,2,...,2,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2.0,1.0,False
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
2,0,1,2,1,1,2,2,0,2,1,...,1,1,2,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
3,0,0,0,0,0,0,0,0,0,2,...,1,2,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2.0,1.0,False
4,2,1,1,2,1,1,0,1,2,2,...,1,2,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96656,0,0,1,0,0,0,0,0,0,2,...,2,2,1,2,0,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
96657,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
96658,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
96659,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2.0,1.0,False


In [34]:
df[df['winner'] == 1]

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
2,0,1,2,1,1,2,2,0,2,1,...,1,1,2,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
5,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
7,0,0,0,0,0,0,0,0,0,0,...,2,2,1,2,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
9,0,0,0,0,0,0,0,0,0,0,...,0,2,2,1,1,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96652,1,1,1,2,2,0,2,2,2,2,...,1,1,1,2,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
96656,0,0,1,0,0,0,0,0,0,2,...,2,2,1,2,0,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
96657,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False
96658,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1.0,2.0,False


In [8]:
#df.loc[2, 'board_series']

In [35]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')
columns2.append('winner')

df_actions = pd.DataFrame(columns=columns2)
working_set = df[df['is_stale'] == False]

preload = []

for i in tqdm(working_set.index):
    series = working_set.loc[i, 'board_series'].copy()
    winning_player = working_set.loc[i, 'winner']
    if len(series) > 3:
        if winning_player == 1:
            for j in range(2, len(series), 2):
                if len(series) > j + 1:
                    flat_board = working_set.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                    #print(len(flat_board))
                    seen = np.resize(working_set.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                    action = np.resize(working_set.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                    result = action - seen
                    target = np.unravel_index(result.argmax(), result.shape)[1]
                    flat_board.append(target)
                    flat_board.append(winning_player)
                    #print(flat_board)
                    preload.append(flat_board)
                    #preload.append(winning_player)
                    #df2 = pd.DataFrame(data=[flat_board], columns=columns2)
        elif winning_player == 2:
            for j in range(1, len(series), 2):
                if len(series) > j + 1:
                    flat_board = working_set.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                    #print(len(flat_board))
                    seen = np.resize(working_set.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                    action = np.resize(working_set.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                    result = action - seen
                    target = np.unravel_index(result.argmax(), result.shape)[1]
                    flat_board.append(target)
                    flat_board.append(winning_player)
                    #print(flat_board)
                    preload.append(flat_board)
                    #preload.append(winning_player)
                    #df2 = pd.DataFrame(data=[flat_board], columns=columns2)

df_actions = pd.DataFrame(data=preload, columns=columns2)

df_actions

100%|██████████████████████████████████████████████████████████████████████████| 91089/91089 [00:32<00:00, 2774.50it/s]


,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,2.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,1,0,0,0,3,2.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,1,1,0,0,2,2.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,2,2,1,1,0,0,4,2.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,2,2,1,1,0,0,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067337,2,0,0,0,0,0,0,2,0,0,...,0,2,0,1,1,0,1,1,1,1.0
1067338,2,0,0,0,0,0,0,2,0,0,...,0,2,1,1,1,0,1,1,1,1.0
1067339,2,0,0,0,0,0,0,2,0,0,...,0,2,1,1,1,0,1,1,3,1.0
1067340,2,0,0,0,0,0,0,2,2,0,...,0,2,1,1,1,0,1,1,3,1.0


In [36]:
#columns5 = [item for item in range(1, 43)]
#columns5.append('winner')
#temp = df_actions.copy()
#temp.loc[:,columns5] = temp.loc[:,columns5].replace(1, 50).replace(2,1).replace(50,2)
#temp
#df_actions = pd.concat([df_actions, temp])

In [40]:
df_actions.to_csv('N:\\connectxactions\\full_actions.csv')

In [37]:
df_actions['winner'].unique()

array([2., 1.])

In [38]:
df_actions['tick'] = 1

In [39]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')
columns2.append('winner')
df_actions_set_one = df_actions.groupby(columns2)['tick'].sum().sort_values(ascending=False).reset_index()
df_actions_set_one['tick_layer_two'] = 1
df_actions_set_one
#df_actions_set_one.groupby(columns2)['tick_layer_two'].sum().sort_values(ascending=False)

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,target,winner,tick,tick_layer_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1.0,7800,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,2.0,5080,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1.0,3844,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,2.0,2172,1
4,0,0,0,0,0,0,0,0,0,0,...,0,2,1,0,0,0,5,1.0,1702,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606363,0,0,0,0,0,0,0,0,0,0,...,1,1,2,0,2,0,1,2.0,1,1
606364,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,0,0,5,2.0,1,1
606365,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,0,0,3,2.0,1,1
606366,0,0,0,0,0,0,0,0,0,0,...,2,2,1,0,1,0,6,1.0,1,1


In [15]:
df_actions_set_one[0:200]

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,target,winner,tick,tick_layer_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,3,2,14853,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1,14853,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,2,9608,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,3,1,9608,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,3,2,6883,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,1,1,810,1
196,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,1,2,810,1
197,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,1,1,810,1
198,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,4,1,806,1


In [16]:
df_actions_set_two = df_actions_set_one.reset_index()
df_actions_set_two = df_actions_set_two.drop(columns=['index'])

columns3 = [item for item in range(1, 43)]
columns3.append('winner')

df_actions_set_two = df_actions_set_two.drop_duplicates(subset=columns3, keep="first").reset_index()
df_actions_set_two = df_actions_set_two.drop(columns=['index','tick', 'tick_layer_two'])
df_actions_set_two

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873445,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,2,2,1,0,1,2
1873446,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,2,1,1,0,2,2
1873447,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,2,1,1,0,2,2
1873448,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,2,2,1,0,3,2


In [17]:
df_actions_set_two.to_csv('N:\\connectxactions\\actions.csv')

In [18]:
df_actions_set_two.to_json('N:\\connectxactions\\actions.json')

In [19]:
sub_template = ''
with open('N:\\connectxactions\\bot_template.txt') as f:
    sub_template = f.read()

cavdata = []
with open('N:\\connectxactions\\actions.csv') as f:
    cavdata = f.read()



In [20]:
from io import StringIO

In [21]:
TESTDATA = StringIO("""col1;col2;col3
    1;4.4;99
    2;4.5;200
    3;4.7;65
    4;3.2;140
    """)

pd.read_csv(TESTDATA, sep=";")

,col1,col2,col3
0,1,4.4,99
1,2,4.5,200
2,3,4.7,65
3,4,3.2,140


In [22]:
TESTDATA = StringIO(cavdata)
columns4 = [item for item in range(1, 43)]
columns4.append('winner')

pd.read_csv(TESTDATA, index_col = columns4)

Unnamed: 0  \
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 winner               
0 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2                0   
                                                                                    2  0  0  0  0  0  0  1  0  0  0  1                1   
                                                                                    0  0  0  0  0  0  0  1  0  0  0  2                2   
                                                                                                         2  0  0  0  1                3   
                                          1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2                4   
...                                                                                                                                 ...   
                                    1  0  1  1  1  0  0  2  0  2  2  2  0  0  2  0  1  2  1  0  0  1  0  2  2  1  0  2          1873445   
                                                                              1  1  2  2  2  0  1  1  2  2  1  1  0  2          1873446   
                                                                                 0  2  2  2  0  1  1  0  2  1  1  0  2          1873447   
                                                                                          1  0  1  1  2  2  2  1  0  2          1873448   
2 2 2 1 2 2 0 2 1 2  1  2  1  0  1  1  2  1  1  1  0  1  2  1  2  2  1  0  2  2  1  1  2  2  0  1  2  1  2  2  1  1  1          1873449   

                                                                                                                             target  
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 winner          
0 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2            3  
                                                                                    2  0  0  0  0  0  0  1  0  0  0  1            3  
                                                                                    0  0  0  0  0  0  0  1  0  0  0  2            3  
                                                                                                         2  0  0  0  1            3  
                                          1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  2            3  
...                                                                                                                             ...  
                                    1  0  1  1  1  0  0  2  0  2  2  2  0  0  2  0  1  2  1  0  0  1  0  2  2  1  0  2            1  
                                                                              1  1  2  2  2  0  1  1  2  2  1  1  0  2            2  
                                                                                 0  2  2  2  0  1  1  0  2  1  1  0  2            2  
                                                                                          1  0  1  1  2  2  2  1  0  2            3  
2 2 2 1 2 2 0 2 1 2  1  2  1  0  1  1  2  1  1  1  0  1  2  1  2  2  1  0  2  2  1  1  2  2  0  1  2  1  2  2  1  1  1            6  

[1873450 rows x 2 columns]

In [23]:
sub_file = sub_template.replace('<REPLACEMEWITHDATA>',f"""
TESTDATA = StringIO('''{cavdata}''')
columns4 = [item for item in range(1, 43)]
columns4.append('winner')
df_map = pd.read_csv(TESTDATA, index_col = columns4)
""")


In [24]:
with open('N:\\connectxactions\\sub.py', 'w') as f:
    f.write(sub_file)

In [25]:
sub_template

'from kaggle_environments import evaluate, make, utils\nimport pandas as pd\nimport numpy as np\nimport random\nimport requests\nfrom tqdm import tqdm\nfrom scipy.signal import convolve2d\nimport sys\nfrom io import StringIO\n\n#result = requests.get(\'https://www.kaggleusercontent.com/episodes/45908179.json\')\n#sys.stdout.write(\'googleresult\')\n#sys.stdout.write(str(len(result.text)))\n\n<REPLACEMEWITHDATA>\n\n\nhorizontal_kernel = np.array([[ 1, 1, 1, 1]])\nvertical_kernel = np.transpose(horizontal_kernel)\ndiag1_kernel = np.eye(4, dtype=np.uint8)\ndiag2_kernel = np.fliplr(diag1_kernel)\ndetection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]\n\ndef winning_move(board, player):\n    for kernel in detection_kernels:\n        if (convolve2d(board == player, kernel, mode="valid") == 4).any():\n            return True\n    return False\n\ndef smart_move(board, me, enemey, recurse_count = 0, target_chosen=0):\n    sys.stdout.write(f\'me:{me} enemy:{enemey}\